In [ ]:
import ipyvuetify as v
import ipywidgets as widgets
from utils import utils
from scripts import run_sepal_process
from scripts import results
import matplotlib.pyplot as plt


In [ ]:
#for testing 
#year = '2020'
#asset_name = "users/bornToBeAlive/aoi_SN"

In [ ]:
#Messages
NO_INPUT = "No input have been selected, provide them through step 1."
SEPAL_MESSAGE = 'click on "Run Postprocessing" to launch the process on your SEPAL account'

In [ ]:
#alerts
sepal_process_output = v.Alert(children=[SEPAL_MESSAGE], type='info', text=True, class_="mt-5")

In [ ]:
#result map 
from scripts import mapping

m_result = mapping.init_result_map()

In [ ]:
#results

sepal_process_hist = v.Flex(xs12=True, lg6=True, class_='pa-0', children=['no result yet'])
sepal_process_result = v.Layout(
    Row=True,
    xs12=True, 
    align_center=True,
    class_='pa-0 mt-5', 
    children=[
        sepal_process_hist,
        v.Flex(xs12=True, lg6=True, class_='pa-0', children=[m_result])
    ]
)

In [ ]:
def create_download_btn(text, url="#"):
    return v.Btn(
        class_='ma-2',
        xs5=True,
        color='primary',
        href=url,
        children=[
            v.Icon(left=True, children=['mdi-download']),
            text
        ]
    )

sepal_download_csv = create_download_btn('Download .csv distribution')
sepal_download_tif = create_download_btn('Download .tif alerts tile')
sepal_download_hist = create_download_btn('Download hist in .png')

In [ ]:
#validate the selected data

sepal_process_btn = v.Btn(
    color='primary', 
    children=[
        v.Icon(left=True, children=['mdi-cogs']),
        'Run Postprocessing'
    ]
)

def sepal_on_click(widget, event, data):
    
    global asset_name
    global year
    global sepal_process_output
    global sepal_process_result
    
    if asset_name == None or year == None:
        utils.displayIO(sepal_process_output, 'error', NO_INPUT)
        return 
    
    utils.toggleLoading(sepal_process_btn)
    #run the process
    alerts_link, stats_link = run_sepal_process.run_sepal_process(asset_name, year, sepal_process_output)
    #display the graph
    figure, png_link, dist_link = results.display_results(asset_name, year)
    sepal_process_hist.children = [figure]
    results.display_alerts(asset_name, year, m_result)
    utils.toggleLoading(sepal_process_btn) 
    
    #edit the download links 
    sepal_download_tif.href= alerts_link
    sepal_download_csv.href= dist_link
    sepal_download_hist.href = png_link

sepal_process_btn.on_event('click', sepal_on_click)

In [ ]:
#create the display 
inputs = v.Layout(
    _metadata={'mount-id': 'data-input'},
    row=True,
    class_="pa-5",
    align_center=True, 
    children=[
        v.Flex(xs12=True, children=[sepal_process_btn]),
        v.Flex(xs12=True, children=[sepal_process_output])
    ]
)

SEPAL_content_main =  v.Layout(
    _metadata={'mount_id': 'widget_sepal_process'},
    Row=True,
    xs12=True,
    align_center=True, 
    class_="ma-5 d-none",
    children=[
        v.Card( 
            class_="pa-5",
            raised=True,
            xs12=True,
            children=[
                v.Html(xs12=True, tag='h2', children=['Step 3: Postprocess']),
                v.Flex(xs12=True, children=[inputs]),   
            ]
        ) 
    ]
)

SEPAL_results = v.Layout(
    _metadata={'mount_id': 'widget_result'},
    Row=True,
    xs12=True,
    align_center=True,
    class_="ma-5 d-none",
    children=[
        v.Card( 
            class_="pa-5",
            raised=True,
            xs12=True,
            children=[
                v.Html(xs12=True, tag='h2', children=['Results']),
                v.Layout(Row=True, xs12=True, children=[
                    sepal_download_csv,
                    sepal_download_tif,
                    sepal_download_hist
                ]),
                sepal_process_result  
            ]
        )
    ]
)
#SEPAL_content_main.class_='ma-5 d-inline' #for debugging
#SEPAL_results.class_='ma-5 d-inline' #for debugging
SEPAL_results
SEPAL_content_main